In [ ]:
from dotenv import load_dotenv
from agent import Agent
load_dotenv()

react = Agent(llm="gpt-4o-mini", system_prompt="I am a helpful AI assistant.")
# react = Agent(llm="llama 3.3 70B", system_prompt="I am a helpful AI assistant.")
response = react.generate_response(prompt="What is the capital of Taiwan?")
print(response)

In [4]:
planner_system_prompt = """You are a Planner Agent designed to reason through complex, open-ended, or ambiguous questions by constructing, reflecting on, and expanding a directed acyclic graph (DAG) of interrelated sub-questions. Your task is not simply to retrieve answers, but to actively explore the question space, refine your understanding, and make informed decisions about when the original question has been sufficiently addressed.

---

## Problem Space Representation: The Question DAG

The DAG is your evolving internal model of the problem. It represents your reasoning process — how the main question relates to sub-questions, intermediate knowledge, and reflections.
Each node contains:
- `node_id`: a unique identifier
- `question`: a sub-question or original question
- `annotation`: your current thoughts, insights, summaries, or hypotheses about that question

Each annotation helps build and maintain your internal representation of the problem. For example:
- A node’s `annotation` may include:
  - A summary of what you currently understand about the question
  - A hypothesis or assumption you are testing
  - A brief note on what you still need to find out
- An `edge_annotation` should briefly explain how the sub-question contributes to answering the parent question — e.g., cause-effect, component, condition, clarification, definition, comparison, or implication.
---

## Input Format

You are always shown the current DAG in JSON format, including all nodes and edges, representing the most up-to-date state of your reasoning process.

---

## Key Reasoning Guidelines

- You cannot delete nodes or edges. Even if a previous path turns out to be incorrect or irrelevant, leave it intact and revise your understanding through `update()`. This mimics how humans preserve earlier lines of thought for traceability, reflection, and learning from missteps.
- You are encouraged to **revisit and revise** previous thoughts using `update`, especially as new information or sub-answers emerge.
- When decomposing, focus on asking the right questions — use logical, causal, definitional, or investigative angles that deepen your understanding.
- When unsure or the question is broad, **start by clarifying or framing the problem**, not jumping to answers.
- For vague or ill-defined questions, take initiative to deconstruct ambiguity, identify what is missing, and reframe as needed. You shape the problem space.

---

## Your Tools

You have three core actions to build and navigate the problem space:

1. **question_decompose**
   Use this to break down a question node into one or more meaningful sub-questions.
   - You may decompose multiple nodes at once.
   - Specify `parent_question_id`, `sub_question`, and an `edge_annotation` explaining the logical or conceptual relationship.
   - Multiple parents pointing to the same sub-question are allowed.
   - Keep the graph acyclic.

   Example:
   ```json
   {
     "graph": [
       {
         "parent_question_id": "Q",
         "sub_question": "How has telework affected work-life boundaries?",
         "edge_annotation": "Understanding personal impact helps assess broader social shifts."
       },
       {
         "parent_question_id": "Q.1",
         "sub_question": "Does telework reinforce or reduce social inequality?",
         "edge_annotation": "Social impact includes distributional effects across groups."
       }
     ]
   }
    ```

2.  **update**
    Use this to revise or expand the annotation of existing nodes.
    - This reflects new insights, summaries, clarifications, or changes in understanding.
    - You are encouraged to use this tool to reflect, correct, or reframe — especially after learning something new.
    - This is a key part of your **metacognitive behavior** — thinking about your thinking.
    
    Example:
    ```json
    {
      "nodes": [
        {
          "question_id": "Q.1",
          "new_annotation": "Workers report blurred boundaries between home and work, leading to both flexibility and stress."
        },
        {
          "question_id": "Q.2",
          "new_annotation": "Emerging evidence suggests that higher-income workers benefit more from telework options, widening inequality."
        }
      ]
    }
    ```

3.  **final_answer**
    Use this only when you believe the original question has been sufficiently addressed **given the available steps so far**.  
    You do not need perfect certainty — you must simply provide a reason why the current DAG gives you enough understanding to form a meaningful answer.
    - Provide a justification explaining why you believe your DAG now contains enough understanding.
    - Your answer should be clear, comprehensive, and informative—sufficient in length to convey key insights.
    - You may use paragraph or bullet point format as appropriate.
    - Aim to include key aspects uncovered in the DAG — such as causes, mechanisms, consequences, or trade-offs — without repeating every detail.
    
    Example:
    ```json
    {
      "reason": "The sub-questions cover key social dimensions — lifestyle, geography, and inequality — and their annotations provide sufficient insight.",
    }
    ```

---

## Metacognitive Expectations

This is not a static search task — it is an evolving thinking process.

- Use `update()` to **reflect**, summarize new insights, question assumptions, or refine your current framing.
- Use `question_decompose()` to **expand the problem space**, identify what needs to be known, or clarify uncertainty.
- Use `final_answer()` only when your internal model (the DAG) gives you enough confidence that you can answer well.
- At each step, treat the DAG as your evolving internal model of understanding — be thoughtful about how you build it.

- When starting from a single root question with no sub-questions yet, you may choose to either:
  - Use `update()` to record your initial thoughts, assumptions, or possible lines of inquiry, or
  - Use `question_decompose()` to begin breaking down the problem into more specific components.
There is no fixed preference — use your best judgment based on the question’s clarity and complexity."""

question = """{
  "nodes": [
    {
      "node_id": "Q",
      "question": "what other technology so dramatically changed society faster than the internet?",
      "annotation": ""
    }
  ],
  "edges": []
}"""
import instructor
from pydantic import BaseModel, Field
from groq import Groq
import os
from dotenv import load_dotenv

load_dotenv()

# Define the tool schema
tool_schema = {
    "name": "get_weather_info",
    "description": "Get the weather information for any location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The location for which we want to get the weather information (e.g., New York)"
            }
        },
        "required": ["location"]
    }
}

# Define the Pydantic model for the tool call
class ToolCall(BaseModel):
    input_text: str = Field(description="The user's input text")
    tool_name: str = Field(description="The name of the tool to call")
    tool_parameters: str = Field(description="JSON string of tool parameters")

class ResponseModel(BaseModel):
    tool_calls: ToolCall

# Patch Groq() with instructor
client = instructor.from_groq(Groq(api_key=os.environ.get("GROQ_API_KEY")), mode=instructor.Mode.JSON)

def run_conversation(question):
    # Prepare the messages
    messages = [
        {
            "role": "system",
            "content": planner_system_prompt
        },
        {
            "role": "user",
            "content": question,
        }
    ]

    # Make the Groq API call
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        response_model=ResponseModel,
        messages=messages,
        parallel_tool_calls=False,
    )

    return response.tool_calls

# Example usage
tool_calls = run_conversation(question)

print(tool_calls)
print(f"Input: {tool_calls.input_text}")
print(f"Tool: {tool_calls.tool_name}")
print(f"Parameters: {tool_calls.tool_parameters}")


input_text='what other technology so dramatically changed society faster than the internet?' tool_name='question_decompose' tool_parameters='{"graph":[{"parent_question_id":"Q","sub_question":"How did the printing press change society?","edge_annotation":"Comparison of societal impact"},{"parent_question_id":"Q","sub_question":"What were the effects of the industrial revolution on society?","edge_annotation":"Comparison of industrial and technological impacts"}]}'
Input: what other technology so dramatically changed society faster than the internet?
Tool: question_decompose
Parameters: {"graph":[{"parent_question_id":"Q","sub_question":"How did the printing press change society?","edge_annotation":"Comparison of societal impact"},{"parent_question_id":"Q","sub_question":"What were the effects of the industrial revolution on society?","edge_annotation":"Comparison of industrial and technological impacts"}]}


In [10]:
from dotenv import load_dotenv
from agent_llama import Agent_llama
load_dotenv()

ll = Agent_llama(llm="llama 3.3 70B")

fake = """{
  "graph": [
    {
      "parent_question_id": "Q",
      "sub_question": "What were the key factors that contributed to the rapid societal change caused by the internet?",
      "edge_annotation": "Understanding these factors can help identify similar technologies."
    },
    {
      "parent_question_id": "Q",
      "sub_question": "Are there other technologies that have had a comparable impact on society, even if not as rapid?",
      "edge_annotation": "Comparing the internet's impact to other technologies can provide perspective."
    }
  ]
}"""
conversations = [{"role": "system", "content": planner_system_prompt},
                {"role": "user", "content": question},
                {"role": "assistant", "content": fake},
                {"role": "user", "content": "Assuming you have enough information; please organize the information you have gathered and write a complete and comprehensive answer to the original question."}]

response = ll.generate_response(conversations=conversations, final_answer_flag=True)

print(response)
# print(response.tool_name)
# print(response.tool_parameters)
print(response.final_answer)

final_answer="\n   The internet has been a pivotal force in reshaping modern society, with its unparalleled speed of adoption and profound impact on how we communicate, work, and live. When comparing the internet to other technologies that have dramatically changed society, several candidates come to mind, including the printing press, electricity, the steam engine, the railroad, the telephone, and television. Each of these technologies led to significant societal shifts, albeit often over a longer period than the internet.\n\n   The printing press, introduced by Johannes Gutenberg in the 15th century, revolutionized the dissemination of knowledge, contributing to the Protestant Reformation and the Enlightenment. It took centuries for the full impact of the printing press to unfold, but it laid the groundwork for modern education, literature, and science.\n\n   Electricity, first harnessed in the late 19th century, transformed urban life, enabling the widespread use of electric lightin